In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
!ls "drive/My Drive/Train_data"

cherry	strawberry  tomato


In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [5]:
!pip install imutils

  Running setup.py bdist_wheel for imutils ... - \ done
  Stored in directory: /root/.cache/pip/wheels/4c/ff/aa/824fb9efc5b8c740d54cd8bc19c7e85fbb8d115c77e56812c7
Successfully built imutils


In [0]:
import numpy as np
import random
import cv2
from imutils import paths
import os
import tensorflow.keras as K
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.image import img_to_array, ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Convolution2D, MaxPooling2D, Flatten, Dense, Dropout)

SEED = 309
np.random.seed(SEED)
random.seed(SEED)

Set parameters for your run here. 

In [0]:
BATCH_SIZE = 128
CLASSES = 3
EPOCHS = 5
# Total train dataset size is 3755
t_size = [3500, 200]
WIDTH, HEIGHT = 300, 300
image_size = (WIDTH, HEIGHT, 3)
test_data_dir = "./drive/My Drive/Train_data"

This collects the indices of the images data ()their file paths)

In [0]:
ps_up = sorted(list(paths.list_images("./drive/My Drive/Train_data")))
ps_up = [ps.replace("\\","") for ps in ps_up]

# build a range as pointers for image files
temp = range(0, len(ps_up))

Load training data

In [0]:
# randomly sample for our training set
# needs to be big enough to be have capacity for a validation set
train_indices = np.random.choice(temp, size = t_size[0], replace = False)
train = [ps_up[i] for i in train_indices]

lbls, imgs = [],[]
for imagePath in train:
  image = cv2.imread(imagePath)
  # load the image, pre-process it, and store it in the data list
  image = cv2.resize(image, (WIDTH, HEIGHT))
  image = img_to_array(image)
  imgs.append(image)
  label = imagePath.split(os.path.sep)[-2]
  lbls.append(label)

X_train = np.array(imgs, dtype="float") / 255.0
y_train = np.array(lbls)

# Binarize the labels
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)

Load test data

In [0]:
# exclude those training images from our test set
temp = [i for i in temp if i not in train]
# subset for testing
test_indices = np.random.choice(temp, size = t_size[1], replace = False)
test = [ps_up[i] for i in test_indices]

lbls, imgs = [],[]
for imagePath in test:
  image = cv2.imread(imagePath)
  # load the image, pre-process it, and store it in the data list\
  image = cv2.resize(image, (WIDTH, HEIGHT))
  image = img_to_array(image)
  imgs.append(image)
  label = imagePath.split(os.path.sep)[-2]
  lbls.append(label)

X_test = np.array(imgs, dtype="float") / 255.0
y_test = np.array(lbls)

# Binarize the labels
# Re use the original binariser.
y_test = lb.fit_transform(y_test)

This is a quick sanity check

In [6]:
print('X_train', 'y_train', 'X_test', 'y_test')
print([len(X_train)==t_size[0],len(y_train)==t_size[0]],
      [len(X_test)==t_size[1], len(y_test)==t_size[1]])

[2750, 2750] [200, 200]


model

In [0]:
classifier = Sequential()
# Base layer
classifier.add(
    Convolution2D(
        64, 3, 3,
        input_shape=image_size,
        activation='relu'
    )
)
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Convolution2D(128, 3, 3, activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Flatten())
classifier.add(Dense(units=300, activation='relu'))
classifier.add(Dense(units=3, activation='sigmoid'))

classifier.compile(
    loss=K.losses.categorical_crossentropy,
    optimizer=K.optimizers.Adam(),
    metrics=['accuracy'],
    
)

Fit without generator:

Callback is CSVLogger so it can write to drive.

In [11]:
classifier.fit(
    X_train, y_train,
    batch_size=32,
    epochs=50,
    verbose=1,
    validation_split=0.2, 
    callbacks=K.callbacks.CSVLogger('./drive/My Drive/prj/logs/', separator=',', append=False)
)
score = classifier.evaluate(X_test, y_test, verbose=0)
score

Train on 2800 samples, validate on 700 samples
Epoch 1/20
2800/2800 [==============================] - 9s 3ms/step - loss: 0.1992 - acc: 0.9282 - val_loss: 0.9413 - val_acc: 0.7357
Epoch 2/20
2800/2800 [==============================] - 9s 3ms/step - loss: 0.1209 - acc: 0.9575 - val_loss: 1.1180 - val_acc: 0.7286
Epoch 3/20
2800/2800 [==============================] - 9s 3ms/step - loss: 0.0725 - acc: 0.9789 - val_loss: 1.1354 - val_acc: 0.7143
Epoch 4/20
2800/2800 [==============================] - 9s 3ms/step - loss: 0.0335 - acc: 0.9936 - val_loss: 1.3108 - val_acc: 0.7186
Epoch 5/20
2800/2800 [==============================] - 9s 3ms/step - loss: 0.0184 - acc: 0.9964 - val_loss: 1.7713 - val_acc: 0.6957
Epoch 6/20
2800/2800 [==============================] - 9s 3ms/step - loss: 0.0565 - acc: 0.9843 - val_loss: 1.4023 - val_acc: 0.7186
Epoch 7/20
2800/2800 [==============================] - 9s 3ms/step - loss: 0.0254 - acc: 0.9943 - val_loss: 1.6430 - val_acc: 0.7157
Epoch 8/20
2800

[0.5157422641804442, 0.94]

Save Model

In [0]:
# classifier.save("./drive/My Drive/prj/models/first.h5", 'w+')

Image noise generator:

In [0]:
transformer= ImageDataGenerator(
    zoom_range=0.2,  # randomly zoom into images
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=True  # randomly flip images
)

Train with generator

In [9]:
classifier.fit_generator(
    transformer.flow(X_train, y_train),
    epochs=10,
    verbose=1 # , callbacks=K.callbacks.CSVLogger('./drive/My Drive/prj/logs/', separator=',', append=False)
)
score_permu = classifier.evaluate(X_test, y_test, verbose=0)
score_permu

Epoch 1/10
86/86 [==============================] - 59s 685ms/step - loss: 1.0576 - acc: 0.4337
Epoch 2/10
86/86 [==============================] - 57s 664ms/step - loss: 0.9180 - acc: 0.5739
Epoch 3/10
86/86 [==============================] - 57s 658ms/step - loss: 0.8582 - acc: 0.6091
Epoch 4/10
86/86 [==============================] - 56s 656ms/step - loss: 0.8284 - acc: 0.6236
Epoch 5/10
86/86 [==============================] - 57s 663ms/step - loss: 0.7824 - acc: 0.6521
Epoch 6/10
86/86 [==============================] - 58s 679ms/step - loss: 0.7574 - acc: 0.6657
Epoch 7/10
86/86 [==============================] - 58s 676ms/step - loss: 0.7194 - acc: 0.7001
Epoch 8/10
86/86 [==============================] - 59s 683ms/step - loss: 0.6786 - acc: 0.7159
Epoch 9/10
86/86 [==============================] - 58s 675ms/step - loss: 0.6564 - acc: 0.7240
Epoch 10/10
86/86 [==============================] - 58s 673ms/step - loss: 0.6422 - acc: 0.7280


[0.750736837387085, 0.635]

In [0]:
# classifier.save("./drive/My Drive/prj/models/first.h5", 'w+')